# Teil 4: Föderiertes Lernen mit Modellmittelung

** Rückblick: ** In Teil 2 dieses Tutorials haben wir ein Modell mit einer sehr einfachen Version von Federated Learning trainiert. Dies erforderte, dass jeder Dateneigentümer dem Modellbesitzer vertraute, um seine Verläufe sehen zu können.

** Beschreibung: ** In diesem Lernprogramm wird gezeigt, wie die erweiterten Aggregationstools aus Teil 3 verwendet werden, damit die Gewichte von einem vertrauenswürdigen "sicheren Mitarbeiter" aggregiert werden können, bevor das endgültige resultierende Modell an den Modellbesitzer zurückgesendet wird (uns).

Auf diese Weise kann nur der sichere Arbeiter sehen, wessen Gewichte von wem stammen. Wir können möglicherweise feststellen, welche Teile des Modells sich geändert haben, aber wir wissen NICHT, welcher Mitarbeiter (Bob oder Alice) welche Änderung vorgenommen hat, wodurch eine Ebene der Privatsphäre entsteht.

Autoren:
  - Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
  - Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
  
Übersetzer:
- Vineet Jain - Github: [@vineetjai](https://github.com/vineetjai)



In [1]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

# Schritt 1: Dateneigentümer erstellen

Zuerst erstellen wir zwei Dateneigentümer (Bob und Alice) mit jeweils einer kleinen Datenmenge. Wir werden auch einen sicheren Computer namens "secure_worker" initialisieren. In der Praxis kann dies sichere Hardware (wie Intels SGX) oder einfach ein vertrauenswürdiger Vermittler sein.

In [2]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

# Schritt 2: Erstellen Sie unser Modell

In diesem Beispiel werden wir mit einem einfachen linearen Modell trainieren. Wir können es normal mit dem nn.Linear-Konstruktor von PyTorch initialisieren.

In [3]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

# Schritt 3: Senden Sie eine Kopie des Modells an Alice und Bob

Als nächstes müssen wir Alice und Bob eine Kopie des aktuellen Modells senden, damit sie Lernschritte für ihre eigenen Datensätze ausführen können.

In [4]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

# Schritt 4: Trainiere Bobs und Alices Modelle (parallel)

Wie bei Federated Learning über Secure Averaging üblich, trainiert jeder Dateneigentümer sein Modell zunächst lokal für mehrere Iterationen, bevor die Modelle zusammen gemittelt werden.

In [5]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.2026) Alice:tensor(0.1496)
Bob:tensor(0.0725) Alice:tensor(0.0861)
Bob:tensor(0.0391) Alice:tensor(0.0714)
Bob:tensor(0.0282) Alice:tensor(0.0594)
Bob:tensor(0.0228) Alice:tensor(0.0495)
Bob:tensor(0.0192) Alice:tensor(0.0412)
Bob:tensor(0.0163) Alice:tensor(0.0343)
Bob:tensor(0.0139) Alice:tensor(0.0285)
Bob:tensor(0.0119) Alice:tensor(0.0237)
Bob:tensor(0.0101) Alice:tensor(0.0198)


# Schritt 5: Senden Sie beide aktualisierten Modelle an einen sicheren Mitarbeiter

Jetzt, da jeder Dateneigentümer über ein teilweise geschultes Modell verfügt, ist es an der Zeit, diese auf sichere Weise zusammen zu mitteln. Dies erreichen wir, indem wir Alice und Bob anweisen, ihr Modell an den sicheren (vertrauenswürdigen) Server zu senden.

Beachten Sie, dass diese Verwendung unserer API bedeutet, dass jedes Modell DIREKT an den Secure_worker gesendet wird. Wir sehen es nie.

In [6]:
alices_model.move(secure_worker)

In [7]:
bobs_model.move(secure_worker)

# Schritt 6: Durchschnitt der Modelle

Schließlich besteht der letzte Schritt für diese Trainingsepoche darin, die trainierten Modelle von Bob und Alice zusammen zu mitteln und dann die Werte für unser globales "Modell" festzulegen.

In [8]:
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())



# Spülen und wiederholen

Und jetzt müssen wir das nur noch mehrmals wiederholen!

In [10]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0023) Alice:tensor(0.0103)
Bob:tensor(0.0005) Alice:tensor(0.0047)
Bob:tensor(8.8708e-05) Alice:tensor(0.0021)
Bob:tensor(5.4481e-05) Alice:tensor(0.0010)
Bob:tensor(0.0001) Alice:tensor(0.0005)
Bob:tensor(0.0002) Alice:tensor(0.0002)
Bob:tensor(0.0002) Alice:tensor(0.0001)
Bob:tensor(0.0002) Alice:tensor(5.7441e-05)
Bob:tensor(0.0002) Alice:tensor(3.0624e-05)
Bob:tensor(0.0001) Alice:tensor(1.6988e-05)


Zuletzt möchten wir sicherstellen, dass unser resultierendes Modell korrekt gelernt wurde, damit wir es anhand eines Testdatensatzes bewerten können. In diesem Spielzeugproblem werden wir die Originaldaten verwenden, aber in der Praxis möchten wir neue Daten verwenden, um zu verstehen, wie gut sich das Modell auf unsichtbare Beispiele verallgemeinert.

In [11]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [12]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0370],
        [0.0326],
        [0.9583],
        [0.9540]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0063)


In diesem Spielzeugbeispiel ist das gemittelte Modell im Vergleich zu einem lokal trainierten Klartextmodell unterpassend. Wir konnten es jedoch trainieren, ohne die Trainingsdaten der einzelnen Mitarbeiter offenzulegen. Wir konnten auch die aktualisierten Modelle von jedem Mitarbeiter auf einem vertrauenswürdigen Aggregator aggregieren, um Datenlecks an den Modellbesitzer zu verhindern.

In einem zukünftigen Tutorial werden wir versuchen, unsere vertrauenswürdige Aggregation direkt mit den Verläufen durchzuführen, damit wir das Modell mit besseren Gradientenschätzungen aktualisieren und zu einem stärkeren Modell gelangen können.

# Herzliche Glückwünsche!!! - Zeit, der Community beizutreten!

Herzlichen Glückwunsch zum Abschluss dieses Notebook-Tutorials! Wenn Ihnen dies gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun!

### Star PySyft auf GitHub

Der einfachste Weg, unserer Community zu helfen, besteht darin, die Repos in der Hauptrolle zu spielen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Mach mit bei unserem Slack!

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei!

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Code-Mitwirkender zu werden! Sie können jederzeit zur Seite PySyft GitHub Issues gehen und nach "Projekten" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "gute erste Ausgabe" gekennzeichnet sind.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)



### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebasis beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups!

[Open Collective Open Collective Page](https://opencollective.com/openmined)